<a href="https://colab.research.google.com/github/Niharika-Saha/ML-hackathon-Group-1/blob/main/HMMCount_w_rl_not_working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Step 1: Install required packages
!pip install hmmlearn
!pip install numpy
!pip install matplotlib
!pip install seaborn
# Step 2: Import libraries
import numpy as np
import string
from collections import defaultdict, Counter
import matplotlib.pyplot as plt
import seaborn as sns
from hmmlearn import hmm
import itertools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 kB 4.2 MB/s eta 0:00:00


In [2]:
# Add this cell after your imports section in HMMCount.ipynb

# =============================================================================
# RL COMPONENTS INTEGRATION FROM rl.ipynb
# =============================================================================

# Additional imports for RL components
import torch
import torch.nn as nn
import torch.optim as optim
import pickle
import random
from collections import deque
import math
from pathlib import Path
import os

# RL Utilities and Environment
class HangmanRLEnvironment:
    def __init__(self, words, max_lives=6, hmm_model=None):
        self.words = [w.strip().upper() for w in words if w.strip()]
        self.max_lives = max_lives
        self.hmm_model = hmm_model
        self.reset()

    def reset(self, word=None):
        self.target_word = word or random.choice(self.words)
        self.masked_word = ['_'] * len(self.target_word)
        self.guessed_letters = set()
        self.wrong_letters = set()
        self.lives = self.max_lives
        self.done = False
        return self._get_state()

    def _get_state(self):
        # Convert game state to numerical representation for RL
        state = []

        # Positional information (26 letters * 20 positions max)
        max_len = 20
        for pos in range(max_len):
            for letter in string.ascii_uppercase:
                if pos < len(self.masked_word) and self.masked_word[pos] == letter:
                    state.append(1.0)  # Letter revealed
                elif letter in self.guessed_letters:
                    state.append(-1.0) # Letter guessed but not here
                else:
                    state.append(0.0)  # Letter not guessed

        # Global letter information
        for letter in string.ascii_uppercase:
            if letter in self.guessed_letters:
                state.append(-1.0)
            else:
                state.append(0.0)

        # Game progress information
        state.append(len(self.guessed_letters) / 26.0)
        state.append(self.lives / self.max_lives)
        state.append(self.masked_word.count('_') / max(1, len(self.masked_word)))

        return np.array(state, dtype=np.float32)

    def step(self, action):
        letter = string.ascii_uppercase[action]
        reward = -0.05  # Small step penalty
        info = {}

        if letter in self.guessed_letters:
            return self._get_state(), reward - 2.0, self.done, {"repeated": True}

        self.guessed_letters.add(letter)

        if letter in self.target_word:
            # Update masked word
            for i, char in enumerate(self.target_word):
                if char == letter:
                    self.masked_word[i] = letter

            # Positive reward for correct guess
            letters_revealed = sum(1 for char in self.target_word if char == letter)
            reward += letters_revealed * 2

            # Check if game is won
            if '_' not in self.masked_word:
                reward += 5.0
                self.done = True
                info["win"] = True
        else:
            self.wrong_letters.add(letter)
            self.lives -= 1
            reward -= 1.0
            info["wrong"] = True

            if self.lives <= 0:
                reward -= 3.0
                self.done = True
                info["lose"] = True

        return self._get_state(), reward, self.done, info

# DQN Network
class DQN(nn.Module):
    def __init__(self, input_dim, hidden_dims=[256, 128], output_dim=26):
        super(DQN, self).__init__()
        layers = []
        prev_dim = input_dim

        for hidden_dim in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, hidden_dim),
                nn.ReLU(),
                nn.Dropout(0.1)
            ])
            prev_dim = hidden_dim

        layers.append(nn.Linear(prev_dim, output_dim))
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

# DQN Agent
class DQNAgent:
    def __init__(self, state_dim, action_dim=26, lr=1e-3, gamma=0.99, epsilon=1.0,
                 epsilon_min=0.01, epsilon_decay=0.995, memory_size=10000):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.lr = lr
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay

        # Neural networks
        self.policy_net = DQN(state_dim)
        self.target_net = DQN(state_dim)
        self.target_net.load_state_dict(self.policy_net.state_dict())

        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=lr)
        self.memory = deque(maxlen=memory_size)
        self.loss_fn = nn.MSELoss()

    def act(self, state, available_actions):
        if random.random() <= self.epsilon:
            return random.choice(available_actions)
        else:
            with torch.no_grad():
                state_tensor = torch.FloatTensor(state).unsqueeze(0)
                q_values = self.policy_net(state_tensor).squeeze().numpy()

                # Mask unavailable actions
                masked_q = q_values.copy()
                for action in range(self.action_dim):
                    if action not in available_actions:
                        masked_q[action] = -np.inf

                return np.argmax(masked_q)

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def replay(self, batch_size=32):
        if len(self.memory) < batch_size:
            return

        batch = random.sample(self.memory, batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)

        states = torch.FloatTensor(states)
        actions = torch.LongTensor(actions)
        rewards = torch.FloatTensor(rewards)
        next_states = torch.FloatTensor(next_states)
        dones = torch.BoolTensor(dones)

        current_q_values = self.policy_net(states).gather(1, actions.unsqueeze(1))
        next_q_values = self.target_net(next_states).max(1)[0].detach()
        target_q_values = rewards + (self.gamma * next_q_values * ~dones)

        loss = self.loss_fn(current_q_values.squeeze(), target_q_values)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # Update epsilon
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def update_target_network(self):
        self.target_net.load_state_dict(self.policy_net.state_dict())

    def save(self, filepath):
        torch.save({
            'policy_state_dict': self.policy_net.state_dict(),
            'target_state_dict': self.target_net.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'epsilon': self.epsilon
        }, filepath)

    def load(self, filepath):
        checkpoint = torch.load(filepath)
        self.policy_net.load_state_dict(checkpoint['policy_state_dict'])
        self.target_net.load_state_dict(checkpoint['target_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        self.epsilon = checkpoint['epsilon']

# Q-Learning Agent
class QLearningAgent:
    def __init__(self, alpha=0.1, gamma=0.99, epsilon=1.0, epsilon_decay=0.995, epsilon_min=0.01):
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min
        self.q_table = defaultdict(lambda: np.zeros(26))

    def get_state_key(self, masked_word, guessed_letters, lives):
        # Create a simplified state representation for Q-learning
        revealed = ''.join([c if c != '_' else '?' for c in masked_word])
        guessed_str = ''.join(sorted(guessed_letters))
        return f"{revealed}_{guessed_str}_{lives}"

    def act(self, state_key, available_actions):
        if random.random() < self.epsilon:
            return random.choice(available_actions)
        else:
            q_values = self.q_table[state_key]
            masked_q = q_values.copy()
            for action in range(26):
                if action not in available_actions:
                    masked_q[action] = -np.inf
            return np.argmax(masked_q)

    def update(self, state, action, reward, next_state, done):
        current_q = self.q_table[state][action]
        if done:
            target = reward
        else:
            target = reward + self.gamma * np.max(self.q_table[next_state])

        self.q_table[state][action] += self.alpha * (target - current_q)

        # Decay epsilon
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def save(self, filepath):
        with open(filepath, 'wb') as f:
            pickle.dump(dict(self.q_table), f)

    def load(self, filepath):
        with open(filepath, 'rb') as f:
            self.q_table = defaultdict(lambda: np.zeros(26), pickle.load(f))

# RL-Enhanced Hybrid Model
class RLEnhancedHybridModel:
    def __init__(self, hybrid_hmm_model, dqn_path='dqn.pt', q_table_path='q_table.pkl',
                 use_dqn=True, use_qtable=True, rl_weight=0.3):
        self.hybrid_model = hybrid_hmm_model
        self.use_dqn = use_dqn
        self.use_qtable = use_qtable
        self.rl_weight = rl_weight

        # Load trained RL models
        self.dqn_agent = None
        self.q_agent = None

        if use_dqn:
            try:
                self.dqn_agent = DQNAgent(state_dim=546)  # Adjust based on your state dimension
                self.dqn_agent.load(dqn_path)
                print("✅ DQN agent loaded successfully")
            except Exception as e:
                print(f"⚠️  Failed to load DQN: {e}")
                self.use_dqn = False

        if use_qtable:
            try:
                self.q_agent = QLearningAgent()
                self.q_agent.load(q_table_path)
                print("✅ Q-table agent loaded successfully")
            except Exception as e:
                print(f"⚠️  Failed to load Q-table: {e}")
                self.use_qtable = False

        print(f"🎯 RL-Enhanced Hybrid Model initialized with RL weight: {rl_weight}")

    def get_letter_probabilities(self, masked_word, guessed_letters):
        # Get HMM probabilities
        hmm_probs = self.hybrid_model.get_letter_probabilities(masked_word, guessed_letters)

        if not (self.use_dqn or self.use_qtable):
            return hmm_probs

        # Get RL probabilities
        rl_probs = self._get_rl_probabilities(masked_word, guessed_letters)

        if not rl_probs:
            return hmm_probs

        # Combine HMM and RL probabilities
        combined_probs = {}
        available_letters = [l for l in string.ascii_uppercase if l not in guessed_letters]

        for letter in available_letters:
            hmm_prob = hmm_probs.get(letter, 0)
            rl_prob = rl_probs.get(letter, 0)

            # Weighted combination
            combined_prob = (1 - self.rl_weight) * hmm_prob + self.rl_weight * rl_prob
            combined_probs[letter] = combined_prob

        # Normalize
        total = sum(combined_probs.values())
        if total > 0:
            combined_probs = {k: v/total for k, v in combined_probs.items()}

        return combined_probs

    def _get_rl_probabilities(self, masked_word, guessed_letters):
        available_actions = [string.ascii_uppercase.index(l) for l in string.ascii_uppercase
                           if l not in guessed_letters]

        if not available_actions:
            return {}

        rl_scores = {}

        # Get DQN scores
        if self.use_dqn and self.dqn_agent:
            # Create state representation (simplified)
            state = np.zeros(546)  # Adjust based on your state dimension
            # You would need to implement proper state encoding here

            try:
                dqn_action = self.dqn_agent.act(state, available_actions)
                for action in available_actions:
                    # Simplified: higher score for DQN's preferred action
                    score = 1.0 if action == dqn_action else 0.1
                    letter = string.ascii_uppercase[action]
                    rl_scores[letter] = rl_scores.get(letter, 0) + score
            except Exception as e:
                print(f"DQN inference error: {e}")

        # Get Q-table scores
        if self.use_qtable and self.q_agent:
            state_key = self.q_agent.get_state_key(''.join(masked_word), guessed_letters, 6)
            try:
                q_action = self.q_agent.act(state_key, available_actions)
                for action in available_actions:
                    q_value = self.q_agent.q_table[state_key][action]
                    letter = string.ascii_uppercase[action]
                    rl_scores[letter] = rl_scores.get(letter, 0) + max(0, q_value)
            except Exception as e:
                print(f"Q-table inference error: {e}")

        # Convert to probabilities
        if rl_scores:
            total_score = sum(rl_scores.values())
            return {letter: score/total_score for letter, score in rl_scores.items()}

        return {}

# Function to load RL models
def load_rl_models(dqn_path='dqn.pt', q_table_path='q_table.pkl'):
    """Load pre-trained RL models"""
    dqn_agent = None
    q_agent = None

    # Load DQN
    try:
        if os.path.exists(dqn_path):
            dqn_agent = DQNAgent(state_dim=546)
            dqn_agent.load(dqn_path)
            print("✅ DQN model loaded successfully")
        else:
            print("⚠️  DQN model file not found")
    except Exception as e:
        print(f"❌ Error loading DQN: {e}")

    # Load Q-table
    try:
        if os.path.exists(q_table_path):
            q_agent = QLearningAgent()
            q_agent.load(q_table_path)
            print("✅ Q-table loaded successfully")
        else:
            print("⚠️  Q-table file not found")
    except Exception as e:
        print(f"❌ Error loading Q-table: {e}")

    return dqn_agent, q_agent

# Enhanced evaluation with RL
def rl_enhanced_evaluation(hybrid_model, test_words, num_games=2000, use_rl=True):
    """Enhanced evaluation using RL-enhanced model"""

    if use_rl:
        print("🚀 Initializing RL-Enhanced Model...")
        model = RLEnhancedHybridModel(hybrid_model)
    else:
        print("🔧 Using Standard Hybrid Model (No RL)")
        model = hybrid_model

    return official_evaluation(model, test_words, num_games)

print("=" * 70)
print("🤖 RL COMPONENTS SUCCESSFULLY INTEGRATED")
print("=" * 70)
print("Available components:")
print("✅ HangmanRLEnvironment - RL training environment")
print("✅ DQNAgent - Deep Q-Network agent")
print("✅ QLearningAgent - Tabular Q-learning agent")
print("✅ RLEnhancedHybridModel - HMM + RL combined model")
print("✅ load_rl_models() - Function to load pre-trained RL models")
print("✅ rl_enhanced_evaluation() - Enhanced evaluation with RL")
print("\\nUsage: rl_enhanced_evaluation(hybrid_model, test_words)")

🤖 RL COMPONENTS SUCCESSFULLY INTEGRATED
Available components:
✅ HangmanRLEnvironment - RL training environment
✅ DQNAgent - Deep Q-Network agent
✅ QLearningAgent - Tabular Q-learning agent
✅ RLEnhancedHybridModel - HMM + RL combined model
✅ load_rl_models() - Function to load pre-trained RL models
✅ rl_enhanced_evaluation() - Enhanced evaluation with RL
\nUsage: rl_enhanced_evaluation(hybrid_model, test_words)


In [3]:
# Step 3: Load and preprocess the corpus
def load_and_preprocess_corpus(file_path):
    """
    Load the corpus and preprocess words
    - Convert to uppercase
    - Remove words with non-alphabet characters
    - Group by word length
    """
    with open(file_path, 'r') as f:
        words = [line.strip().upper() for line in f if line.strip()]

    # Filter only alphabetic words
    words = [word for word in words if word.isalpha()]

    # Group words by length
    words_by_length = defaultdict(list)
    for word in words:
        words_by_length[len(word)].append(word)

    print(f"Total words after preprocessing: {len(words)}")
    print(f"Word length distribution:")
    for length in sorted(words_by_length.keys()):
        print(f"  Length {length}: {len(words_by_length[length])} words")

    return words, words_by_length

# Load the corpus (upload your corpus.txt first)
words, words_by_length = load_and_preprocess_corpus('corpus.txt')

Total words after preprocessing: 49979
Word length distribution:
  Length 1: 46 words
  Length 2: 84 words
  Length 3: 388 words
  Length 4: 1169 words
  Length 5: 2340 words
  Length 6: 3755 words
  Length 7: 5111 words
  Length 8: 6348 words
  Length 9: 6787 words
  Length 10: 6465 words
  Length 11: 5452 words
  Length 12: 4292 words
  Length 13: 3094 words
  Length 14: 2019 words
  Length 15: 1226 words
  Length 16: 698 words
  Length 17: 375 words
  Length 18: 174 words
  Length 19: 88 words
  Length 20: 40 words
  Length 21: 16 words
  Length 22: 8 words
  Length 23: 3 words
  Length 24: 1 words


In [4]:
# Step 4: Analyze letter frequencies and patterns
def analyze_corpus(words_by_length):
    """
    Analyze letter frequencies and positional distributions
    """
    # Overall letter frequency
    all_letters = ''.join([''.join(words) for words in words_by_length.values()])
    letter_freq = Counter(all_letters)

    print("Overall letter frequency (top 10):")
    for letter, freq in letter_freq.most_common(10):
        print(f"  {letter}: {freq} ({freq/len(all_letters)*100:.2f}%)")

    # Positional frequency analysis for different lengths
    positional_freq = defaultdict(lambda: defaultdict(int))

    for length, word_list in words_by_length.items():
        for word in word_list:
            for pos, letter in enumerate(word):
                positional_freq[length][(pos, letter)] += 1

    return letter_freq, positional_freq

letter_freq, positional_freq = analyze_corpus(words_by_length)

Overall letter frequency (top 10):
  E: 49203 (10.37%)
  A: 42089 (8.87%)
  I: 42047 (8.86%)
  O: 35808 (7.54%)
  R: 33577 (7.07%)
  N: 33314 (7.02%)
  T: 32191 (6.78%)
  S: 29044 (6.12%)
  L: 27406 (5.77%)
  C: 21718 (4.58%)


In [5]:
# Step 5: Prepare HMM training data for each word length
def prepare_hmm_training_data(words_by_length, min_words_threshold=5):
    """
    Prepare training data for HMMs for each word length
    Convert words to numerical sequences for HMM training

    Parameters:
    - words_by_length: Dictionary of words grouped by length
    - min_words_threshold: Minimum number of words required to train HMM for a given length
    """
    # Create letter to index mapping (A=0, B=1, ..., Z=25)
    letters = string.ascii_uppercase
    letter_to_idx = {letter: idx for idx, letter in enumerate(letters)}

    training_data = {}

    # Print available word lengths for debugging
    available_lengths = sorted(words_by_length.keys())
    print(f"Available word lengths: {available_lengths}")
    print(f"Lengths 22, 23, 24 present: {22 in words_by_length}, {23 in words_by_length}, {24 in words_by_length}")

    for length, word_list in words_by_length.items():
        # Include all lengths including 22, 23, 24 if they have enough words
        if len(word_list) < min_words_threshold:
            print(f"Skipping length {length}: only {len(word_list)} words (need at least {min_words_threshold})")
            continue

        sequences = []
        for word in word_list:
            # Convert word to numerical sequence
            seq = [letter_to_idx[char] for char in word]
            sequences.append(seq)

        training_data[length] = np.array(sequences)

        print(f"Length {length}: {len(sequences)} sequences")

    # Specifically check and report on lengths 22, 23, 24
    for length in [22, 23, 24]:
        if length in training_data:
            print(f"✓ INCLUDED - Length {length}: {len(training_data[length])} sequences")
        elif length in words_by_length:
            print(f"✗ EXCLUDED - Length {length}: {len(words_by_length[length])} words (below threshold)")
        else:
            print(f"✗ NOT FOUND - Length {length}: No words in corpus")

    return training_data, letter_to_idx

# Run with lower threshold to include more word lengths
training_data, letter_to_idx = prepare_hmm_training_data(words_by_length, min_words_threshold=1)

Available word lengths: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
Lengths 22, 23, 24 present: True, True, True
Length 11: 5452 sequences
Length 6: 3755 sequences
Length 9: 6787 sequences
Length 16: 698 sequences
Length 14: 2019 sequences
Length 10: 6465 sequences
Length 8: 6348 sequences
Length 12: 4292 sequences
Length 13: 3094 sequences
Length 5: 2340 sequences
Length 18: 174 sequences
Length 4: 1169 sequences
Length 3: 388 sequences
Length 7: 5111 sequences
Length 15: 1226 sequences
Length 17: 375 sequences
Length 22: 8 sequences
Length 19: 88 sequences
Length 2: 84 sequences
Length 1: 46 sequences
Length 20: 40 sequences
Length 21: 16 sequences
Length 23: 3 sequences
Length 24: 1 sequences
✓ INCLUDED - Length 22: 8 sequences
✓ INCLUDED - Length 23: 3 sequences
✓ INCLUDED - Length 24: 1 sequences


In [6]:
# Step 6: HYBRID CFP + Count-based HMM Training
class HybridHangmanModel:
    """
    Combines the best of CFP (accuracy) and HMM (generalization)
    """
    def __init__(self, corpus_words, training_data, letter_to_idx, alpha=0.3):
        self.cfp = CandidateFilterOracle(corpus_words)
        self.count_hmms = train_count_hmm_models(training_data)
        self.letter_to_idx = letter_to_idx
        self.letters = string.ascii_uppercase
        self.alpha = alpha  # Weight for HMM (0-1), CFP weight = 1-alpha

        print(f"🎯 Hybrid Model Initialized: CFP weight={1-alpha:.1f}, HMM weight={alpha:.1f}")

    def get_letter_probabilities(self, masked_word, guessed_letters):
        """
        Combine CFP and Count-based HMM probabilities
        """
        # Get CFP probabilities (high accuracy)
        cfp_probs = self.cfp.get_letter_probabilities(masked_word, guessed_letters)

        # Get HMM probabilities (good generalization)
        hmm_probs = self._get_hmm_probabilities(masked_word, guessed_letters)

        # If CFP has no candidates, rely more on HMM
        if not cfp_probs or max(cfp_probs.values()) == 0:
            return hmm_probs

        # If HMM failed, rely on CFP
        if not hmm_probs:
            return cfp_probs

        # Combine probabilities
        hybrid_probs = {}
        for letter in self.letters:
            if letter not in guessed_letters:
                cfp_prob = cfp_probs.get(letter, 0)
                hmm_prob = hmm_probs.get(letter, 0)

                # Dynamic weighting: if CFP is confident, trust it more
                cfp_confidence = max(cfp_probs.values())
                dynamic_alpha = self.alpha * (1 - cfp_confidence)  # Less HMM weight if CFP is confident

                hybrid_prob = (1 - dynamic_alpha) * cfp_prob + dynamic_alpha * hmm_prob
                hybrid_probs[letter] = hybrid_prob

        # Normalize
        total = sum(hybrid_probs.values())
        if total > 0:
            hybrid_probs = {letter: prob/total for letter, prob in hybrid_probs.items()}

        return hybrid_probs

    def _get_hmm_probabilities(self, masked_word, guessed_letters):
        """Get probabilities from count-based HMM"""
        word_length = len(masked_word)

        if word_length not in self.count_hmms:
            return {}

        model = self.count_hmms[word_length]

        # Prepare observations
        observations = []
        for char in masked_word:
            if char == '_':
                observations.append(-1)  # Missing
            else:
                observations.append(self.letter_to_idx[char])

        try:
            # Get posteriors using count-based HMM
            posteriors = model.forward_backward(observations)

            # Aggregate probabilities for blank positions
            letter_probs = np.zeros(26)
            for pos, char in enumerate(masked_word):
                if char == '_':
                    for letter_idx in range(26):
                        letter = self.idx_to_letter(letter_idx)
                        if letter not in guessed_letters:
                            letter_probs[letter_idx] += posteriors[pos, letter_idx]

            # Normalize
            if np.sum(letter_probs) > 0:
                letter_probs /= np.sum(letter_probs)

            return {self.idx_to_letter(i): letter_probs[i]
                    for i in range(26) if self.idx_to_letter(i) not in guessed_letters}

        except Exception as e:
            return {}

    def idx_to_letter(self, idx):
        """Convert index to letter"""
        return chr(65 + idx)

# COUNT-BASED HMM IMPLEMENTATION (from previous)
class CountBasedHMM:
    def __init__(self):
        self.transition_probs = None
        self.emission_probs = None
        self.initial_probs = None

    def train(self, sequences, alpha=0.1):
        n_states = 26

        # Initialize counts with smoothing
        transition_counts = np.ones((n_states, n_states)) * alpha
        emission_counts = np.ones((n_states, n_states)) * alpha
        initial_counts = np.ones(n_states) * alpha

        # Count transitions and emissions
        for seq in sequences:
            if len(seq) > 0:
                initial_counts[seq[0]] += 1

            for i in range(len(seq)):
                emission_counts[seq[i], seq[i]] += 1
                if i < len(seq) - 1:
                    transition_counts[seq[i], seq[i+1]] += 1

        # Convert to probabilities
        self.initial_probs = initial_counts / np.sum(initial_counts)
        self.transition_probs = transition_counts / np.sum(transition_counts, axis=1, keepdims=True)
        self.emission_probs = emission_counts / np.sum(emission_counts, axis=1, keepdims=True)

    def forward_backward(self, observations):
        n_positions = len(observations)
        n_states = 26

        # Forward pass
        forward = np.zeros((n_positions, n_states))
        for state in range(n_states):
            if observations[0] == -1:
                forward[0, state] = self.initial_probs[state]
            else:
                forward[0, state] = self.initial_probs[state] * self.emission_probs[state, observations[0]]

        for t in range(1, n_positions):
            for j in range(n_states):
                sum_prob = 0
                for i in range(n_states):
                    sum_prob += forward[t-1, i] * self.transition_probs[i, j]

                if observations[t] == -1:
                    forward[t, j] = sum_prob
                else:
                    forward[t, j] = sum_prob * self.emission_probs[j, observations[t]]

        # Backward pass
        backward = np.ones((n_positions, n_states))
        for t in range(n_positions-2, -1, -1):
            for i in range(n_states):
                sum_prob = 0
                for j in range(n_states):
                    if observations[t+1] == -1:
                        emission_prob = 1.0
                    else:
                        emission_prob = self.emission_probs[j, observations[t+1]]
                    sum_prob += self.transition_probs[i, j] * emission_prob * backward[t+1, j]
                backward[t, i] = sum_prob

        # Combine
        posteriors = forward * backward
        posteriors = posteriors / np.sum(posteriors, axis=1, keepdims=True)
        return posteriors


def train_count_hmm_models(training_data):
    """Train count-based HMMs for all word lengths"""
    count_hmms = {}

    print("🚀 Training Count-based HMMs for Hybrid Model...")
    for length, sequences in training_data.items():
        if len(sequences) < 1:
            continue

        print(f"   📊 Length {length}: {len(sequences)} sequences")
        model = CountBasedHMM()
        model.train(sequences, alpha=0.1)
        count_hmms[length] = model

    print(f"✅ Trained {len(count_hmms)} count-based HMMs")
    return count_hmms

# CFP ORACLE (from previous)
class CandidateFilterOracle:
    def __init__(self, corpus_words):
        self.by_len = defaultdict(list)
        for word in corpus_words:
            w = word.strip().upper()
            if w and w.isalpha():
                self.by_len[len(w)].append(w)

    @staticmethod
    def matches_mask(word, mask, wrong_letters):
        if len(word) != len(mask):
            return False
        if any(letter in word for letter in wrong_letters):
            return False
        for wc, mc in zip(word, mask):
            if mc != '_' and wc != mc:
                return False
        return True

    def get_letter_probabilities(self, masked_word, guessed_letters):
        word_length = len(masked_word)
        revealed_letters = set(char for char in masked_word if char != '_')
        wrong_letters = guessed_letters - revealed_letters

        # Get candidate words
        candidates = [
            word for word in self.by_len.get(word_length, [])
            if self.matches_mask(word, masked_word, wrong_letters)
        ]

        if not candidates:
            return self._fallback_probs(guessed_letters)

        # Count letter frequencies in blank positions
        blank_positions = [i for i, char in enumerate(masked_word) if char == '_']
        letter_counts = Counter()

        for word in candidates:
            for pos in blank_positions:
                letter_counts[word[pos]] += 1

        # Convert to probabilities
        total_count = sum(letter_counts.values())
        prob_dict = {}
        for letter in string.ascii_uppercase:
            if letter not in guessed_letters:
                prob_dict[letter] = letter_counts[letter] / total_count if total_count > 0 else 0

        # Normalize
        total_prob = sum(prob_dict.values())
        if total_prob > 0:
            prob_dict = {letter: prob/total_prob for letter, prob in prob_dict.items()}
        else:
            prob_dict = self._fallback_probs(guessed_letters)

        return prob_dict

    def _fallback_probs(self, guessed_letters):
        available = [l for l in string.ascii_uppercase if l not in guessed_letters]
        prob = 1.0 / len(available) if available else 0
        return {letter: prob for letter in available}

# 🚀 INITIALIZE HYBRID MODEL
print("🎯 Initializing Hybrid CFP + Count-based HMM Model...")
hybrid_model = HybridHangmanModel(words, training_data, letter_to_idx, alpha=0.3)

print("\n" + "="*60)
print("🏆 HYBRID MODEL READY!")
print("="*60)
print("✅ CFP Oracle: Word-based candidate filtering")
print("✅ Count-based HMM: Statistical pattern learning")
print("✅ Dynamic weighting: Adaptive confidence-based blending")
print("✅ All word lengths supported")
print(f"✅ Initial blend: {70}% CFP + {30}% HMM")

# Update your main HMM reference to use the hybrid model
hangman_hmm = hybrid_model

print("\n🎯 Ready for evaluation with 2000 games!")

🎯 Initializing Hybrid CFP + Count-based HMM Model...
🚀 Training Count-based HMMs for Hybrid Model...
   📊 Length 11: 5452 sequences
   📊 Length 6: 3755 sequences
   📊 Length 9: 6787 sequences
   📊 Length 16: 698 sequences
   📊 Length 14: 2019 sequences
   📊 Length 10: 6465 sequences
   📊 Length 8: 6348 sequences
   📊 Length 12: 4292 sequences
   📊 Length 13: 3094 sequences
   📊 Length 5: 2340 sequences
   📊 Length 18: 174 sequences
   📊 Length 4: 1169 sequences
   📊 Length 3: 388 sequences
   📊 Length 7: 5111 sequences
   📊 Length 15: 1226 sequences
   📊 Length 17: 375 sequences
   📊 Length 22: 8 sequences
   📊 Length 19: 88 sequences
   📊 Length 2: 84 sequences
   📊 Length 1: 46 sequences
   📊 Length 20: 40 sequences
   📊 Length 21: 16 sequences
   📊 Length 23: 3 sequences
   📊 Length 24: 1 sequences
✅ Trained 24 count-based HMMs
🎯 Hybrid Model Initialized: CFP weight=0.7, HMM weight=0.3

🏆 HYBRID MODEL READY!
✅ CFP Oracle: Word-based candidate filtering
✅ Count-based HMM: Statistical

In [7]:
# Step 8: Test the HMM with example game states
def test_hmm_predictions():
    """Test HMM predictions with various game states"""

    test_cases = [
        # (masked_word, guessed_letters, description)
        (['_', '_', '_', '_', '_'], set(), "Empty 5-letter word"),
        (['A', '_', '_', '_', '_'], set('A'), "5-letter word starting with A"),
        (['_', '_', '_', 'E', '_'], set('E'), "5-letter word with E at position 3"),
        (['_', '_', '_', '_', '_', '_'], set(), "Empty 6-letter word"),
    ]

    for masked_word, guessed_letters, description in test_cases:
        print(f"\n{description}:")
        print(f"Masked word: {' '.join(masked_word)}")
        print(f"Guessed letters: {''.join(sorted(guessed_letters))}")

        probs = hangman_hmm.get_letter_probabilities(masked_word, guessed_letters)

        # Show top 5 predictions
        top_letters = sorted(probs.items(), key=lambda x: x[1], reverse=True)[:5]
        print("Top 5 letter predictions:")
        for letter, prob in top_letters:
            print(f"  {letter}: {prob:.4f}")

test_hmm_predictions()


Empty 5-letter word:
Masked word: _ _ _ _ _
Guessed letters: 
Top 5 letter predictions:
  A: 0.1073
  E: 0.0981
  O: 0.0694
  R: 0.0676
  I: 0.0634

5-letter word starting with A:
Masked word: A _ _ _ _
Guessed letters: A
Top 5 letter predictions:
  E: 0.1023
  I: 0.0946
  O: 0.0846
  N: 0.0731
  S: 0.0701

5-letter word with E at position 3:
Masked word: _ _ _ E _
Guessed letters: E
Top 5 letter predictions:
  R: 0.1329
  A: 0.0755
  N: 0.0711
  L: 0.0675
  T: 0.0672

Empty 6-letter word:
Masked word: _ _ _ _ _ _
Guessed letters: 
Top 5 letter predictions:
  E: 0.1116
  A: 0.1021
  R: 0.0712
  I: 0.0709
  N: 0.0615


In [8]:
# Step 7: OFFICIAL EVALUATION - 2000 Games with Official Scoring Formula
def official_evaluation(hangman_hmm, test_words, num_games=2000, max_wrong_guesses=6):
    """
    Official evaluation as per problem statement
    Plays 2000 games and calculates the official score
    """
    import numpy as np
    from collections import defaultdict

    print("🎯 OFFICIAL EVALUATION - 2000 HANGMAN GAMES")
    print("=" * 60)
    print(f"Playing {num_games} games with {max_wrong_guesses} wrong guesses allowed per game")
    print()

    # Initialize metrics
    results = {
        'games_won': 0,
        'games_lost': 0,
        'total_wrong_guesses': 0,
        'total_repeated_guesses': 0,
        'games_details': [],
        'score_breakdown': defaultdict(int)
    }

    # Play specified number of games
    for game_id in range(min(num_games, len(test_words))):
        if game_id % 500 == 0:
            print(f"🎮 Played {game_id}/{num_games} games...")

        target_word = test_words[game_id]
        game_result = play_single_game(hangman_hmm, target_word, max_wrong_guesses, game_id + 1)

        # Aggregate results
        if game_result['won']:
            results['games_won'] += 1
        else:
            results['games_lost'] += 1

        results['total_wrong_guesses'] += game_result['wrong_guesses']
        results['total_repeated_guesses'] += game_result['repeated_guesses']
        results['games_details'].append(game_result)

    # Calculate official score
    official_score = calculate_official_score(results, num_games)

    # Print comprehensive results
    print_official_results(results, official_score, num_games)

    return results, official_score

def play_single_game(hangman_hmm, target_word, max_wrong_guesses, game_id):
    """
    Play a single Hangman game and return detailed results
    """
    masked_word = ['_'] * len(target_word)
    guessed_letters = set()
    wrong_guesses = 0
    repeated_guesses = 0
    game_log = []

    # Game loop
    while wrong_guesses < max_wrong_guesses and '_' in masked_word:
        # Get letter probabilities from HMM
        probs = hangman_hmm.get_letter_probabilities(masked_word, guessed_letters)

        if not probs:
            # No probabilities available, use fallback
            available_letters = [chr(i) for i in range(65, 91) if chr(i) not in guessed_letters]
            if not available_letters:
                break
            next_letter = available_letters[0]
        else:
            # Choose letter with highest probability
            next_letter = max(probs.items(), key=lambda x: x[1])[0]

        # Check for repeated guess
        if next_letter in guessed_letters:
            repeated_guesses += 1
            game_log.append(f"Repeated guess: {next_letter}")
            continue

        # Add to guessed letters
        guessed_letters.add(next_letter)

        # Check if letter is in target word
        if next_letter in target_word:
            # Update masked word
            for i, char in enumerate(target_word):
                if char == next_letter:
                    masked_word[i] = next_letter
            game_log.append(f"Correct: {next_letter} -> {' '.join(masked_word)}")
        else:
            wrong_guesses += 1
            game_log.append(f"Wrong: {next_letter} ({wrong_guesses}/{max_wrong_guesses} wrong)")

    # Determine game outcome
    won = '_' not in masked_word
    actual_word = ''.join(target_word)
    guessed_word = ''.join(masked_word)

    return {
        'game_id': game_id,
        'target_word': actual_word,
        'won': won,
        'wrong_guesses': wrong_guesses,
        'repeated_guesses': repeated_guesses,
        'total_guesses': len(guessed_letters),
        'final_word': guessed_word,
        'game_log': game_log
    }

def calculate_official_score(results, num_games):
    """
    Calculate official score using the formula:
    Final Score = (Success Rate * 2000) - (Total Wrong Guesses * 5) - (Total Repeated Guesses * 2)
    """
    success_rate = results['games_won'] / num_games
    total_wrong_guesses = results['total_wrong_guesses']
    total_repeated_guesses = results['total_repeated_guesses']

    score = (success_rate * 2000) - (total_wrong_guesses * 5) - (total_repeated_guesses * 2)

    # Store breakdown for analysis
    results['score_breakdown']['success_component'] = success_rate * 2000
    results['score_breakdown']['wrong_penalty'] = total_wrong_guesses * 5
    results['score_breakdown']['repeated_penalty'] = total_repeated_guesses * 2
    results['score_breakdown']['success_rate'] = success_rate
    results['score_breakdown']['total_games'] = num_games

    return score

def print_official_results(results, official_score, num_games):
    """
    Print comprehensive evaluation results
    """
    print("\n" + "=" * 70)
    print("🏆 OFFICIAL EVALUATION RESULTS")
    print("=" * 70)

    # Basic statistics
    print(f"📊 GAME STATISTICS:")
    print(f"   Total Games Played: {num_games}")
    print(f"   Games Won: {results['games_won']}")
    print(f"   Games Lost: {results['games_lost']}")
    print(f"   Success Rate: {results['games_won']/num_games:.3f} ({results['games_won']}/{num_games})")

    print(f"\n🎯 PERFORMANCE METRICS:")
    print(f"   Total Wrong Guesses: {results['total_wrong_guesses']}")
    print(f"   Total Repeated Guesses: {results['total_repeated_guesses']}")
    print(f"   Average Wrong Guesses per Game: {results['total_wrong_guesses']/num_games:.2f}")
    print(f"   Average Repeated Guesses per Game: {results['total_repeated_guesses']/num_games:.2f}")

    print(f"\n💰 SCORE BREAKDOWN:")
    print(f"   Success Component: {results['score_breakdown']['success_component']:.2f}")
    print(f"   Wrong Guesses Penalty: -{results['score_breakdown']['wrong_penalty']:.2f}")
    print(f"   Repeated Guesses Penalty: -{results['score_breakdown']['repeated_penalty']:.2f}")
    print(f"   FINAL SCORE: {official_score:.2f}")

    # Additional analysis
    print(f"\n📈 ADDITIONAL INSIGHTS:")

    # Word length analysis
    won_by_length = defaultdict(int)
    lost_by_length = defaultdict(int)

    for game in results['games_details']:
        length = len(game['target_word'])
        if game['won']:
            won_by_length[length] += 1
        else:
            lost_by_length[length] += 1

    print(f"   Performance by Word Length:")
    for length in sorted(set(won_by_length.keys()) | set(lost_by_length.keys())):
        won = won_by_length[length]
        total = won + lost_by_length.get(length, 0)
        if total > 0:
            success_rate = won / total
            print(f"     Length {length}: {won}/{total} won ({success_rate:.1%})")

    # Efficiency analysis
    total_correct_guesses = sum(game['total_guesses'] - game['wrong_guesses'] for game in results['games_details'])
    total_guesses = sum(game['total_guesses'] for game in results['games_details'])
    efficiency = total_correct_guesses / total_guesses if total_guesses > 0 else 0

    print(f"   Guess Efficiency: {efficiency:.1%} ({total_correct_guesses}/{total_guesses} correct guesses)")

# Load test words for evaluation
def load_test_words_for_evaluation(file_path, num_words=2000):
    """Load test words for official evaluation"""
    with open(file_path, 'r') as f:
        words = [line.strip().upper() for line in f if line.strip()]

    words = [word for word in words if word.isalpha()]

    if len(words) < num_words:
        print(f"⚠️  Warning: Only {len(words)} test words available, using all of them")
        return words
    else:
        return words[:num_words]

# 🚀 RUN OFFICIAL EVALUATION
print("Loading test words for official evaluation...")
test_words_eval = load_test_words_for_evaluation('test.txt', num_words=2000)

print(f"Loaded {len(test_words_eval)} test words for evaluation")
print("Starting official 2000-game evaluation...")

# Run the official evaluation
results, official_score = official_evaluation(hangman_hmm, test_words_eval, num_games=2000, max_wrong_guesses=6)

# Save detailed results
def save_detailed_results(results, official_score, filename="evaluation_results.txt"):
    """Save detailed evaluation results to file"""
    with open(filename, 'w') as f:
        f.write("OFFICIAL HANGMAN EVALUATION RESULTS\n")
        f.write("=" * 50 + "\n\n")

        f.write(f"FINAL SCORE: {official_score:.2f}\n\n")

        f.write("SUMMARY STATISTICS:\n")
        f.write(f"Games Played: {results['score_breakdown']['total_games']}\n")
        f.write(f"Games Won: {results['games_won']}\n")
        f.write(f"Success Rate: {results['score_breakdown']['success_rate']:.3f}\n")
        f.write(f"Total Wrong Guesses: {results['total_wrong_guesses']}\n")
        f.write(f"Total Repeated Guesses: {results['total_repeated_guesses']}\n\n")

        f.write("SCORE BREAKDOWN:\n")
        f.write(f"Success Component: {results['score_breakdown']['success_component']:.2f}\n")
        f.write(f"Wrong Guesses Penalty: -{results['score_breakdown']['wrong_penalty']:.2f}\n")
        f.write(f"Repeated Guesses Penalty: -{results['score_breakdown']['repeated_penalty']:.2f}\n\n")

        f.write("FIRST 10 GAME RESULTS:\n")
        for game in results['games_details'][:10]:
            status = "WON" if game['won'] else "LOST"
            f.write(f"Game {game['game_id']}: {game['target_word']} -> {status} "
                   f"(Wrong: {game['wrong_guesses']}, Repeated: {game['repeated_guesses']})\n")

    print(f"📄 Detailed results saved to {filename}")

# Save results
save_detailed_results(results, official_score)

# Performance assessment
def assess_performance(score):
    """Assess the performance based on final score"""
    print("\n" + "=" * 50)
    print("📋 PERFORMANCE ASSESSMENT")
    print("=" * 50)

    if score >= 1500:
        assessment = "EXCELLENT 🏆"
        feedback = "Outstanding performance! Your agent is highly efficient."
    elif score >= 1000:
        assessment = "VERY GOOD 🥈"
        feedback = "Strong performance with good success rate and efficiency."
    elif score >= 500:
        assessment = "GOOD 🥉"
        feedback = "Solid performance with room for optimization."
    elif score >= 0:
        assessment = "FAIR 📊"
        feedback = "Basic functionality achieved, needs improvement in efficiency."
    else:
        assessment = "NEEDS WORK 🔧"
        feedback = "Focus on improving success rate and reducing wrong guesses."

    print(f"FINAL SCORE: {score:.2f}")
    print(f"ASSESSMENT: {assessment}")
    print(f"FEEDBACK: {feedback}")

# Final assessment
assess_performance(official_score)

print(f"\n🎉 EVALUATION COMPLETE!")
print(f"📊 Check 'evaluation_results.txt' for detailed game-by-game results")

Loading test words for official evaluation...
Loaded 2000 test words for evaluation
Starting official 2000-game evaluation...
🎯 OFFICIAL EVALUATION - 2000 HANGMAN GAMES
Playing 2000 games with 6 wrong guesses allowed per game

🎮 Played 0/2000 games...
🎮 Played 500/2000 games...
🎮 Played 1000/2000 games...
🎮 Played 1500/2000 games...

🏆 OFFICIAL EVALUATION RESULTS
📊 GAME STATISTICS:
   Total Games Played: 2000
   Games Won: 703
   Games Lost: 1297
   Success Rate: 0.351 (703/2000)

🎯 PERFORMANCE METRICS:
   Total Wrong Guesses: 10213
   Total Repeated Guesses: 0
   Average Wrong Guesses per Game: 5.11
   Average Repeated Guesses per Game: 0.00

💰 SCORE BREAKDOWN:
   Success Component: 703.00
   Wrong Guesses Penalty: -51065.00
   Repeated Guesses Penalty: -0.00
   FINAL SCORE: -50362.00

📈 ADDITIONAL INSIGHTS:
   Performance by Word Length:
     Length 2: 0/2 won (0.0%)
     Length 3: 0/9 won (0.0%)
     Length 4: 4/37 won (10.8%)
     Length 5: 10/91 won (11.0%)
     Length 6: 24/138 w